In [ ]:
import pandas as pd
import re

chord_list = [
    ['0', 'A', '', ''],
    ['1', '', 'A#', 'Bb'],
    ['2', '', 'B', ''],
    ['3', 'C', '', ''],
    ['4', '', 'C#', 'Db'],
    ['5', 'D', '', ''],
    ['6', '', 'D#', 'Eb'],
    ['7', 'E', '', ''],
    ['8', 'F', '', ''],
    ['9', '', 'F#', 'Gb'],
    ['10', 'G', '', ''],
    ['11', '', 'G#', 'Ab']
]
chord_df = pd.DataFrame(chord_list, columns=['key', 'natural', 'sharp', 'flat'])

# コード検出用の正規表現
chord_pattern = re.compile(r'\[([A-Ga-g#b]+[^/\]]*)(/[A-Ga-g#b]*)?\]')

# コードの移調関数
def transpose_chord(chord, semitones, sharp_flag, chord_df):
    root_match = re.match(r'([A-Ga-g#b]+)', chord)
    if not root_match:
        return chord
    root = root_match.group(1)

    # 現在のルートのkey値を取得
    root_row = chord_df[(chord_df['natural'] == root) | (chord_df['sharp'] == root) | (chord_df['flat'] == root)]
    if root_row.empty:
        return chord
    current_key = int(root_row.iloc[0]['key'])

    # 移調後のkey値を計算
    new_key = (current_key + semitones) % 12
    new_row = chord_df[chord_df['key'] == str(new_key)].iloc[0]

    # 移調後のルート名を選択
    if new_row['natural']:
        new_root = new_row['natural']
    else:
        new_root = new_row['sharp'] if sharp_flag == 1 else new_row['flat']

    # ルート以外の部分を保持
    suffix = chord[len(root):]
    return new_root + suffix

# 和音を移調する
def transpose_lyrics(lyrics_text, semitones, sharp_flag, chord_df):
    transposed_lyrics = lyrics_text

    def replace_chord(match):
        chord, bass = match.groups()
        transposed_chord = transpose_chord(chord, semitones, sharp_flag, chord_df)
        if bass:
            transposed_bass = transpose_chord(bass[1:], semitones, sharp_flag, chord_df)
            return f"[{transposed_chord}/{transposed_bass}]"
        return f"[{transposed_chord}]"

    transposed_lyrics = chord_pattern.sub(replace_chord, lyrics_text)
    return transposed_lyrics

